In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive')

In [0]:
from zipfile import ZipFile
with ZipFile('train.zip', 'r') as z:z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile('test.zip', 'r') as z:z.extractall()

In [0]:
x_test=[]
os.chdir('/content/drive/My Drive/test')
import cv2
for i in os.listdir():
    dummy = cv2.imread(i)
    dummy = cv2.resize(dummy,(128,128))
    x_test.append(dummy)

In [0]:
x_train = []
y_train = []

In [0]:
os.chdir('/content/drive/My Drive/train')

In [22]:
os.listdir()

['Fat Hen',
 'Small-flowered Cranesbill',
 'Cleavers',
 'Black-grass',
 'Sugar beet',
 'Shepherds Purse',
 'Charlock',
 'Loose Silky-bent',
 'Scentless Mayweed',
 'Maize',
 'Common Chickweed',
 'Common wheat']

In [23]:
x_train = []
y_train = []
import cv2
for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('/content/drive/My Drive/train/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)

Fat Hen
Small-flowered Cranesbill
Cleavers
Black-grass
Sugar beet
Shepherds Purse
Charlock
Loose Silky-bent
Scentless Mayweed
Maize
Common Chickweed
Common wheat


In [24]:
x_train[0].shape

(128, 128, 3)

In [0]:
import pandas as pd
dum = pd.get_dummies(y_train)

In [0]:
encoded_labels = dum
y_train = dum

In [0]:
import numpy as np
y_train = np.array(y_train)

In [0]:

x_train = np.array(x_train)

In [29]:
x_train[0].shape

(128, 128, 3)

In [30]:
from sklearn.model_selection import train_test_split
x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print (len(x_train2))
print (len(x_val))

3800
950


In [31]:
x_train2[0].shape

(128, 128, 3)

In [0]:
x_train2 = x_train2.reshape(x_train2.shape[0],128,128,3)

In [0]:
x_val = x_val.reshape(x_val.shape[0],128,128,3)

In [34]:
print (x_train2.shape)
print (x_val.shape)

(3800, 128, 128, 3)
(950, 128, 128, 3)


In [35]:
print(y_train2.shape)
print(y_val.shape)

(3800, 12)
(950, 12)


In [36]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense

Using TensorFlow backend.


In [0]:
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam

In [38]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
#model.add(Dropout(0.3))
model.add(Dense(units = 32, activation = 'relu'))
#model.add(Dropout(0.3))
model.add(Dense(units = 12, activation = 'softmax'))

In [39]:
optimizer = adam(lr=0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 128)      

In [41]:
batch_size = 64
nb_epochs = 10
history = model.fit(x_train,y_train,
                    epochs=20, 
                    validation_data=(x_val,y_val),
                    verbose = 1,
                    initial_epoch=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 4750 samples, validate on 950 samples
Epoch 1/20
4750/4750 [==============================] - 14s 3ms/step - loss: 1.8886 - acc: 0.3509 - val_loss: 1.4204 - val_acc: 0.5032
Epoch 2/20
4750/4750 [==============================] - 10s 2ms/step - loss: 1.1578 - acc: 0.5996 - val_loss: 0.8277 - val_acc: 0.6937
Epoch 3/20
4750/4750 [==============================] - 10s 2ms/step - loss: 0.7740 - acc: 0.7269 - val_loss: 0.6181 - val_acc: 0.7853
Epoch 4/20
4750/4750 [==============================] - 10s 2ms/step - loss: 0.5808 - acc: 0.8017 - val_loss: 0.5306 - val_acc: 0.8105
Epoch 5/20
4750/4750 [==============================] - 9s 2ms/step - loss: 0.4158 - acc: 0.8531 - val_loss: 0.4992 - val_acc: 0.8274
Epoch 6/20
4750/4750 [==============================] - 9s 2ms/step - loss: 0.3592 - acc: 0.8728 - val_loss: 0.3671 - val_acc: 0.8632
Epoch 7/20
4750/4750 [===========================

In [42]:
model2 = Sequential()
model2.add(BatchNormalization(input_shape = (128,128,3)))
model2.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
model2.add(MaxPooling2D(pool_size=2))
model2.add(Dropout(0.5))

model2.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2))
model2.add(Dropout(0.5))

model2.add(Convolution2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2))
model2.add(Dropout(0.5))

model2.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2))
model2.add(Dropout(0.5))

model2.add(Flatten()) 

# fully connected layer
model2.add(Dense(units=128,activation = 'relu'))
model2.add(Dense(units = 64, activation = 'relu'))
model2.add(Dropout(0.3))
model2.add(Dense(units = 32, activation = 'relu'))
model2.add(Dropout(0.3))
model2.add(Dense(units = 12, activation = 'softmax')) 
model2.compile(optimizer='sgd', loss = 'categorical_crossentropy',metrics = ['accuracy'])
model2.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 63, 63, 64)        32832     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 64)        0         
_____

In [43]:
batch_size = 128
nb_epochs = 10
history2 = model2.fit(x_train,y_train,batch_size=64,
                    epochs=10, 
                    validation_data=(x_val,y_val),
                    verbose = 1,
                    initial_epoch=0)

Train on 4750 samples, validate on 950 samples
Epoch 1/10
4750/4750 [==============================] - 10s 2ms/step - loss: 2.5437 - acc: 0.0865 - val_loss: 2.4714 - val_acc: 0.1032
Epoch 2/10
4750/4750 [==============================] - 9s 2ms/step - loss: 2.4737 - acc: 0.1051 - val_loss: 2.4640 - val_acc: 0.1389
Epoch 3/10
4750/4750 [==============================] - 9s 2ms/step - loss: 2.4573 - acc: 0.1099 - val_loss: 2.4579 - val_acc: 0.1411
Epoch 4/10
4750/4750 [==============================] - 9s 2ms/step - loss: 2.4517 - acc: 0.1261 - val_loss: 2.4526 - val_acc: 0.1411
Epoch 5/10
4750/4750 [==============================] - 9s 2ms/step - loss: 2.4437 - acc: 0.1347 - val_loss: 2.4485 - val_acc: 0.1411
Epoch 6/10
4750/4750 [==============================] - 9s 2ms/step - loss: 2.4419 - acc: 0.1229 - val_loss: 2.4438 - val_acc: 0.1411
Epoch 7/10
4750/4750 [==============================] - 9s 2ms/step - loss: 2.4321 - acc: 0.1385 - val_loss: 2.4394 - val_acc: 0.1411
Epoch 8/10
475